In [1]:
%matplotlib inline

In [2]:
import json
import os
import sys
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

sys.path.append('../../pygkernels')
from pygkernels.measure import kernels

In [3]:
def calc_avranks(results):  # {dataset: {classifier: accuracy}}
    ranks = defaultdict(list)
    for _, classifier_accuracy in results.items():
        classifiers, accuracies = zip(*list(classifier_accuracy.items()))
        for classifier, rank in zip(classifiers, rankdata(-np.array(accuracies), method='min')):
            ranks[classifier].append(rank)
    ranks = {k: np.mean(v) for k, v in sorted(ranks.items(), key=lambda x: x[0])}
    return list(ranks.values()), list(ranks.keys()), len(results)

In [4]:
kernels_order = [x.name for x in kernels]
list(enumerate(kernels_order))

[(0, 'Katz'),
 (1, 'logKatz'),
 (2, 'For'),
 (3, 'logFor'),
 (4, 'Comm'),
 (5, 'logComm'),
 (6, 'Heat'),
 (7, 'logHeat'),
 (8, 'NHeat'),
 (9, 'logNHeat'),
 (10, 'SCT'),
 (11, 'SCCT'),
 (12, 'RSP'),
 (13, 'FE'),
 (14, 'PPR'),
 (15, 'logPPR'),
 (16, 'ModifPPR'),
 (17, 'logModifPPR'),
 (18, 'HeatPR'),
 (19, 'logHeatPR'),
 (20, 'DF'),
 (21, 'logDF'),
 (22, 'Abs'),
 (23, 'logAbs'),
 (24, 'SP-CT')]

# Leaderboards

In [5]:
# root = '/media/illusionww/68949C3149F4E819/phd/pygkernels/montecarlo_lfr_simple'
# dataset = []
# for fn in tqdm(os.listdir(root)):
#     with open(f'{root}/{fn}', 'rb') as f:
#         dataset.append(json.load(f))
        
with open('filtered_dataset.json', 'r') as f:
    dataset = json.load(f)

In [6]:
idx_maxari_eq1 = [idx for idx, d in enumerate(dataset) if np.max(np.array(list(d['measure_best_results'].values()))) == 1]
idx_maxari_eq1 = set(idx_maxari_eq1)
idx_several_answers = [idx for idx, d in enumerate(dataset) 
                       if np.sum(np.max(np.array(list(d['measure_best_results'].values()))) == np.array(list(d['measure_best_results'].values()))) > 1]
idx_several_answers = set(idx_several_answers)

len(idx_maxari_eq1), 
len(idx_several_answers)

dataset_mari_less1 = [d for idx, d in enumerate(dataset) if idx not in idx_maxari_eq1 and idx not in idx_several_answers]
dataset_mari_eq1 = [d for idx, d in enumerate(dataset) if idx in idx_maxari_eq1]

In [7]:
len(dataset_mari_less1)

4856

In [8]:
len(dataset_mari_eq1)

1984

In [9]:
len([x for x in idx_several_answers if x not in idx_maxari_eq1])

415

In [10]:
len([x for x in idx_maxari_eq1 if x not in idx_several_answers])

504

In [11]:
lb = {}

# Leaderboard by ranks ">="
ranks, names, n_experiments = calc_avranks(dict(enumerate([data['measure_best_results'] for data in dataset])))
ranks_ge = dict(zip(names, ranks))
lb['ranks(>=)'] = ranks_ge

# Leaderboard by wins ">", maxari < 1
wins_ge = {kernel_name: 0 for kernel_name in kernels_order}
for kernel_name in kernels_order:
    for data in dataset_mari_less1:
        item = data['measure_best_results']
        if item[kernel_name] >= np.max([v for k, v in item.items() if k != kernel_name]):
            wins_ge[kernel_name] += 1
lb['wins(>), maxari<1'] = wins_ge
# pd.DataFrame(Counter(ranks_ge).most_common(), columns=['measure', 'rank']).sort_values('rank')

# Leaderboard by wins "maxari == 1"
wins_maxari1 = {kernel_name: 0 for kernel_name in kernels_order}
for kernel_name in kernels_order:
    for data in dataset_mari_eq1:
        item = data['measure_best_results']
        if item[kernel_name] >= np.max([v for k, v in item.items() if k != kernel_name]):
            wins_maxari1[kernel_name] += 1
lb['wins(>=), maxari=1'] = wins_maxari1
            
# # Leaderboard by wins ">"
# wins_gt = {kernel_name: 0 for kernel_name in kernels_order}
# for kernel_name in kernels_order:
#     for data in dataset:
#         item = data['measure_best_results']
#         if item[kernel_name] > np.max([v for k, v in item.items() if k != kernel_name]):
#             wins_gt[kernel_name] += 1
# lb['wins (>)'] = wins_gt
# # pd.DataFrame(Counter(ranks_ge).most_common(), columns=['measure', 'rank']).sort_values('rank')


# # Leaderboard by ranks ">"
# ranks, names, n_experiments = calc_avranks(dict(enumerate([item for item in [data['measure_best_results'] for data in dataset]
#                                                            if np.sum(np.array(list(item.values())) == np.max(list(item.values()))) == 1])))
# ranks_gt = dict(zip(names, ranks))
# lb['ranks (>)'] = ranks_gt
# # pd.DataFrame(zip(names, ranks), columns=['measure', 'rank']).sort_values('rank')


# # Leaderboard by ranks "maxari == 1"
# ranks, names, n_experiments = calc_avranks(dict(enumerate([item for item in [data['measure_best_results'] for data in dataset]
#                                                            if np.max(list(item.values())) == 1])))
# ranks_maxari1 = dict(zip(names, ranks))
# lb['ranks (maxari = 1)'] = ranks_maxari1

In [12]:
df = pd.DataFrame(lb)
df.sort_values('wins(>), maxari<1', ascending=False)

,ranks(>=),"wins(>), maxari<1","wins(>=), maxari=1"
SCCT,4.699518,2272,1734
NHeat,6.949139,815,969
Comm,8.481599,269,697
logComm,12.564025,253,864
RSP,7.632391,182,1348
logDF,14.826051,157,643
SCT,8.328739,141,1232
SP-CT,8.959614,128,1281
logKatz,9.927498,124,1169
Abs,18.635975,86,172


In [13]:
df.to_excel('leaderboard.xlsx')